## Import

In [1]:
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# 시각화
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)
import warnings ; warnings.filterwarnings('ignore')

## Read data

In [2]:
msft, ma, intc, mcd, aapl = pd.read_pickle('data/total_data.pkl')
display(msft)

,Open,High,Low,Close,Volume,log_return,MA5,MA20,UPPER,MID,...,UNRATE,IR,ER,RS,TB,VIX,NASDAQ,OP,SP500,SOX
2014-01-02,37.349998,37.400002,37.099998,31.290831,30632200.0,-0.006705,31.341517,31.319295,32.674710,31.449548,...,6.7,0.69,1.3670,424784.0,-339.861,14.23,4143.07,95.14,1831.979980,527.750000
2014-01-03,37.200001,37.220001,36.599998,31.080309,31134800.0,-0.006751,31.254448,31.296534,32.431733,31.364079,...,6.7,0.69,1.3606,424784.0,-339.861,13.76,4131.91,93.66,1831.369995,526.679993
2014-01-06,36.849998,36.889999,36.110001,30.423506,43603700.0,-0.021359,30.977467,31.213389,32.386083,31.285348,...,6.7,0.69,1.3636,424784.0,-339.861,13.55,4113.68,93.12,1826.770020,523.770020
2014-01-07,36.330002,36.490002,36.209999,30.659273,35802800.0,0.007720,30.871402,31.160616,32.231168,31.203247,...,6.7,0.69,1.3618,424784.0,-339.861,12.92,4153.18,93.31,1837.880005,527.429993
2014-01-08,36.000000,36.139999,35.580002,30.111944,59971700.0,-0.018013,30.618250,31.060742,31.998363,31.079044,...,6.7,0.69,1.3586,424784.0,-339.861,12.87,4165.61,91.90,1837.489990,535.760010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-26,333.720001,336.109985,328.489990,327.241577,21520600.0,-0.019349,332.834887,330.023416,343.705424,332.338908,...,3.5,4.47,1.0910,694415.0,-935.445,14.25,13335.78,69.09,4328.819824,3515.770020
2023-06-27,331.859985,336.149994,329.299988,333.186890,24354100.0,0.018005,332.952221,330.324699,343.788182,332.422560,...,3.5,4.47,1.0961,694415.0,-935.445,13.74,13555.67,67.68,4378.410156,3642.409912
2023-06-28,334.660004,337.980011,333.809998,334.461609,20259500.0,0.003819,333.455350,330.718690,343.987721,332.653601,...,3.5,4.47,1.0904,694415.0,-935.445,13.43,13591.75,69.54,4376.859863,3609.479980
2023-06-29,334.709991,336.109985,332.619995,333.664886,16997000.0,-0.002385,333.525196,330.999280,344.026393,332.985223,...,3.5,4.47,1.0881,694415.0,-935.445,13.54,13591.33,69.85,4396.439941,3614.179932


## Split Train-Test data

In [10]:
# Train : Vaild : Test = 8*0.8 : 8*0.2 : 2로 쪼갠다
def splitData(DATA):
    train, test = DATA.iloc[:int(DATA.shape[0]*0.8)], DATA.iloc[int(DATA.shape[0]*0.8):]
    train, vaild = train.iloc[:int(train.shape[0]*0.8)], train.iloc[int(train.shape[0]*0.8):]
    return train, vaild, test

In [11]:
msft_train, msft_vaild, msft_test = splitData(msft)
ma_train, ma_vaild, ma_test = splitData(ma)
intc_train, intc_vaild, intc_test = splitData(intc)
mcd_train, mcd_vaild, mcd_test = splitData(mcd)
aapl_train, aapl_vaild, aapl_test = splitData(aapl)

print(f'train: {msft_train.index[0]} ~ {msft_train.index[-1]}')
print(f'vaild: {msft_vaild.index[0]} ~ {msft_vaild.index[-1]}')
print(f'test: {msft_test.index[0]} ~ {msft_test.index[-1]}')

train: 2014-01-02 00:00:00 ~ 2020-01-29 00:00:00
vaild: 2020-01-30 00:00:00 ~ 2021-08-05 00:00:00
test: 2021-08-06 00:00:00 ~ 2023-06-30 00:00:00


## Reshape

In [5]:
# 최신 데이터를 위로 하면 좋은지 시도해보기
def ReshapeByStep(DATA, STEP):
    y = DATA['target'] ; DATA.drop('target', axis=1, inplace=True)
    y = y.iloc[STEP:]
    ease = []
    for i in range(DATA.shape[0] - STEP):
        ease.append(DATA[i:i+STEP].values)
    x = np.reshape(np.array(ease), ((DATA.shape[0] - STEP), STEP, DATA.shape[1]))
    return x, y

In [6]:
msft_train_x, msft_train_y = ReshapeByStep(msft_train, 10)

## Auto-encoder
- "Application of Deep Learning to Algorithmic Trading(2017)" 논문을 참고하였다.
- https://chioni.github.io/posts/ae/

In [ ]:
input_window = Input(shape=(window_length,))
encoded = Dense(encoding_dim, activation='relu')(input_window)
decoded = Dense(window_length, activation='softmax')(encoded)
autoencoder = Model(input_window, decoded)
encoder = Model(input_window, encoded)

In [ ]:
autoencoder.summary()

In [ ]:
autoencoder.compile(optimizer='adam', loss='categorical_crossentropy')
history = autoencoder.fit(msft_train_x, msft_train_x,
                          epochs=epochs,
                          batch_size=128,
                          shuffle=False,
                          validation_data=(msft_vaild_x, msft_valid_x))

In [ ]:
decoded_train = autoencoder.predict(msft_valid_x)
le_train = encoder.predict(msft_train_x)
le_test = encoder.predict(msft_test_x)